In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nfl-stadium-attendance-dataset/games.csv
/kaggle/input/nfl-stadium-attendance-dataset/standings.csv
/kaggle/input/nfl-stadium-attendance-dataset/attendance.csv
/kaggle/input/football-events/ginf.csv
/kaggle/input/football-events/dictionary.txt
/kaggle/input/football-events/events.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Nov.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2020-Feb.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Oct.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2019-Dec.csv
/kaggle/input/ecommerce-events-history-in-cosmetics-shop/2020-Jan.csv
/kaggle/input/taylor-concert-tours-impact-on-attendance-and/Taylor_Train.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/nfl-stadium-attendance-dataset/attendance.csv')
train_df.head()

,team,team_name,year,total,home,away,week,weekly_attendance
0,Arizona,Cardinals,2000,893926,387475,506451,1,77434.0
1,Arizona,Cardinals,2000,893926,387475,506451,2,66009.0
2,Arizona,Cardinals,2000,893926,387475,506451,3,NaN
3,Arizona,Cardinals,2000,893926,387475,506451,4,71801.0
4,Arizona,Cardinals,2000,893926,387475,506451,5,66985.0


In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10846 entries, 0 to 10845
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   team               10846 non-null  object 
 1   team_name          10846 non-null  object 
 2   year               10846 non-null  int64  
 3   total              10846 non-null  int64  
 4   home               10846 non-null  int64  
 5   away               10846 non-null  int64  
 6   week               10846 non-null  int64  
 7   weekly_attendance  10208 non-null  float64
dtypes: float64(1), int64(5), object(2)
memory usage: 678.0+ KB


In [4]:
train_df['team_name'].nunique()

32

In [5]:
from category_encoders import MEstimateEncoder

X_encode = train_df.sample(frac=0.25, random_state=42)
y_encode = X_encode['weekly_attendance']
X_pretrain = train_df.drop(X_encode.index)
y_train = X_pretrain['weekly_attendance']

# Create the encoder instance. Choose m to control noise.
encoder = MEstimateEncoder(cols=["team","team_name"], m=5.0)

# Fit the encoder on the encoding split.
encoder.fit(X_encode, y_encode)

# Encode the City and Weather columns to create the final training data
X_train = encoder.transform(X_pretrain)

In [6]:
X_train

,team,team_name,year,total,home,away,week,weekly_attendance
1,60619.540503,60619.540503,2000,893926,387475,506451,2,66009.0
2,60619.540503,60619.540503,2000,893926,387475,506451,3,NaN
4,60619.540503,60619.540503,2000,893926,387475,506451,5,66985.0
5,60619.540503,60619.540503,2000,893926,387475,506451,6,44296.0
6,60619.540503,60619.540503,2000,893926,387475,506451,7,38293.0
...,...,...,...,...,...,...,...,...
10838,73116.813175,73116.813175,2019,1101466,523906,577560,10,NaN
10840,73116.813175,73116.813175,2019,1101466,523906,577560,12,57754.0
10841,73116.813175,73116.813175,2019,1101466,523906,577560,13,71504.0
10842,73116.813175,73116.813175,2019,1101466,523906,577560,14,77296.0


In [7]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
train_df_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
train_df_imputed

,team,team_name,year,total,home,away,week,weekly_attendance
0,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,2.0,66009.000000
1,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,3.0,67637.700431
2,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,5.0,66985.000000
3,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,6.0,44296.000000
4,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,7.0,38293.000000
...,...,...,...,...,...,...,...,...
8129,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,10.0,67637.700431
8130,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,12.0,57754.000000
8131,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,13.0,71504.000000
8132,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,14.0,77296.000000


In [8]:
cor_matrix = train_df_imputed.corr()
cor_matrix

,team,team_name,year,total,home,away,week,weekly_attendance
team,1.000000,0.982003,-0.003448,0.732132,0.752626,0.124632,0.000398,0.355170
team_name,0.982003,1.000000,0.011606,0.728597,0.746256,0.131227,-0.000424,0.354759
year,-0.003448,0.011606,1.000000,0.097796,0.053418,0.140540,0.003602,0.057195
total,0.732132,0.728597,0.097796,1.000000,0.937758,0.407505,-0.005004,0.496684
home,0.752626,0.746256,0.053418,0.937758,1.000000,0.064996,-0.005871,0.465138
away,0.124632,0.131227,0.140540,0.407505,0.064996,1.000000,0.001057,0.204064
week,0.000398,-0.000424,0.003602,-0.005004,-0.005871,0.001057,1.000000,-0.036480
weekly_attendance,0.355170,0.354759,0.057195,0.496684,0.465138,0.204064,-0.036480,1.000000


In [9]:
train_df_imputed

,team,team_name,year,total,home,away,week,weekly_attendance
0,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,2.0,66009.000000
1,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,3.0,67637.700431
2,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,5.0,66985.000000
3,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,6.0,44296.000000
4,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,7.0,38293.000000
...,...,...,...,...,...,...,...,...
8129,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,10.0,67637.700431
8130,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,12.0,57754.000000
8131,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,13.0,71504.000000
8132,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,14.0,77296.000000


In [10]:
X = train_df_imputed.drop(columns=['weekly_attendance'])
y = train_df_imputed[['weekly_attendance']]
train_df_imputed

,team,team_name,year,total,home,away,week,weekly_attendance
0,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,2.0,66009.000000
1,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,3.0,67637.700431
2,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,5.0,66985.000000
3,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,6.0,44296.000000
4,60619.540503,60619.540503,2000.0,893926.0,387475.0,506451.0,7.0,38293.000000
...,...,...,...,...,...,...,...,...
8129,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,10.0,67637.700431
8130,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,12.0,57754.000000
8131,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,13.0,71504.000000
8132,73116.813175,73116.813175,2019.0,1101466.0,523906.0,577560.0,14.0,77296.000000


In [11]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_y,valid_y = train_test_split(X,y)

In [12]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(train_X,train_y)
preds = my_model.predict(valid_X)

In [13]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(valid_y,preds)
mae

6386.085666712758

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3]
}


grid_search = GridSearchCV(estimator=my_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_X, train_y)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters:", best_params)
print("Best score:", best_score)

Best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150}
Best score: -60408401.313432835


In [15]:
best_params = grid_search.best_params_
best_model = XGBRegressor(**best_params)
best_model.fit(train_X, train_y)
pred2 = best_model.predict(valid_X)
mae = mean_absolute_error(valid_y,pred2)
mae

5419.6245892179995

In [16]:
valid_X

,team,team_name,year,total,home,away,week
7665,65035.379842,65035.379842,2018.0,1040509.0,507651.0,532858.0,11.0
7976,65402.785371,65402.785371,2019.0,1070025.0,534794.0,535231.0,2.0
5481,69261.371259,69261.371259,2013.0,1041545.0,514553.0,526992.0,4.0
3637,68085.179567,68085.179567,2009.0,1074077.0,568660.0,505417.0,3.0
2807,73116.813175,73116.813175,2006.0,1236415.0,701049.0,535366.0,1.0
...,...,...,...,...,...,...,...
1813,65402.785371,65402.785371,2004.0,1045505.0,512969.0,532536.0,1.0
6402,69138.535810,69138.535810,2015.0,1154629.0,555868.0,598761.0,16.0
4192,64180.430783,64180.430783,2010.0,1061603.0,504262.0,557341.0,8.0
6497,73116.813175,73116.813175,2015.0,1196087.0,609672.0,586415.0,6.0


In [17]:
final_valid = pd.DataFrame(index = valid_X.index)
final_valid['predicted_attendees'] = pred2
final_valid['actual_attendance'] = valid_y
final_valid

,predicted_attendees,actual_attendance
7665,65719.695312,67683.000000
7976,67014.421875,78416.000000
5481,65840.367188,73118.000000
3637,67014.421875,70950.000000
2807,74813.367188,90608.000000
...,...,...
1813,65840.367188,64105.000000
6402,71071.343750,69596.000000
4192,66412.796875,81395.000000
6497,73079.835938,78160.000000
